Pewien słuchacz szkoły muzycznej ma w sobie niesamowity talent. Jednak przed jej ukończeniem wstrzymuje go jeden przedmiot - "Kompozytorzy muzyki klasycznej". Słuchacz ten, mając dość niepowodzeń w zdawaniu tego tematu, zwraca się do Was o pomoc.

Zadanie polega na stworzeniu modelu rekurencyjnego, który będzie przewidywał kompozytora danego utworu klasycznego w oparciu o jego zapis w formie sekwencji akordów. Akordy znormalizowane zostały do klucza C-dur lub a-moll, w zależności od skali utworu (durowa/molowa).
Dane przygotowane są w postaci pickle (https://docs.python.org/3/library/pickle.html), w których znajduje się lista krotek z sekwencjami i odpowiadającymi im klasami (kompozytorami), odpowiednio: {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}. Dane treningowe znajdują się w pliku train.pkl. W pliku test_no_target.pkl znajdują się testowe sekwencje, dla których predykcje mają Państwo przewidzieć.

Uwaga, utwory mogą mieć różne długości. Do stworzenia batchy dla przykładów różnej długości proszę wykorzystać omówiony na zajęciach padding i trenować z wykorzystaniem wyrównanych tensorów lub spakowanych sekwencji (PackedSequence).

Bardzo proszę, żeby zwrócili Państwo archiwum zip, zgodnie z instrukcjami:
- Archiwum powinno być nazwane {poniedzialek/piatek}_nazwisko1_nazwisko2.zip, bez nawiasów klamrowych przy dniu tygodnia
- W archiwum proszę, bez zbędnych podfolderów, umieścić pliki ze swoim kodem i testowe predykcje nazwane {poniedzialek/piatek}_nazwisko1_nazwisko2.csv (lub nazwa drużyny), bez nawiasów klamrowych przy dniu tygodnia
- Testowe predykcje powinny mieć kolejność zgodną z kolejnością sekwencji w picklu. Plik .csv nie powinien mieć nagłówka ani indeksów.

Proszę zwracać uwagę na prawidłowe nazewnictwo oraz odpowiedni format zwracanych plików. Niedostosowanie się do wytycznych może spowodować nieuwzględnienie Państwa w rankingu i utratę punktów za osiągnięty wynik!
Proszę także o udokumentowanie wykonanych eksperymentów.

In [37]:
import pickle
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [38]:
DATA_PATH = './p5'
VOCAB_SIZE = 128  # Assuming MIDI note numbers (0-127)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 5
BATCH_SIZE = 32
CUDA_LAUNCH_BLOCKING=1

In [39]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cpu


In [40]:
with open(f'{DATA_PATH}/train.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open(f'{DATA_PATH}/test_no_target.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [41]:
class MusicDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        seq, label = self.data[idx]
        return torch.tensor(seq, dtype=torch.long), label

train_dataset = MusicDataset(train_data)
test_dataset = MusicDataset([(seq, -1) for seq in test_data])  # labels are not known for test data

def collate_fn(batch):
    sequences, labels = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    lengths, sorted_idx = torch.sort(torch.tensor(lengths), descending=True)
    sequences_padded = sequences_padded[sorted_idx]
    labels = torch.tensor(labels, dtype=torch.long)[sorted_idx]
    return sequences_padded, labels, lengths


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [42]:
class ComposerClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers=2, bidirectional=True):
        super(ComposerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
    
    def forward(self, x, lengths):
        embedded = self.embedding(x)
        lengths, perm_idx = lengths.sort(0, descending=True)
        embedded = embedded[perm_idx]
        packed = pack_padded_sequence(embedded, lengths, batch_first=True)
        packed_output, (hidden, cell) = self.lstm(packed)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        out = self.fc(hidden)
        return out

model = ComposerClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [43]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for sequences, labels, lengths in loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        predictions = model(sequences, lengths)
        
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    return epoch_loss / len(loader)

N_EPOCHS = 10

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}/{N_EPOCHS}, Train Loss: {train_loss:.4f}')


In [44]:
def predict(model, loader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for sequences, _, lengths in loader:
            sequences = sequences.to(device)
            outputs = model(sequences, lengths)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

test_predictions = predict(model, test_loader, device)

df = pd.DataFrame(test_predictions)
df.to_csv('poniedzialek_kieruczenko_ziarek.csv', header=False, index=False)

IndexError: index out of range in self